# Design messaging service

### Notes
- FB messenger keep all the messages in DB -> can retrieve the messages
- Whatsapp delete the message on DB after the receiver receive the messages
    + Have end-to-end encryption -> 3rd party cant read the messages
- **http Request**
    + Request - Response architecture: client request - server response
    + Server cant request -> No bidirectional connection 
    + Not suitable for chat/messaging service
- Chat messaging architecture
    + **Websoket**: bidirectional connection
    + **BOSH**: bidirectional stream
    + **long poll http**: bidirectional http

### Features
- User status: Online/ Active [x] hours ago
- Friend adding
- Online 1-on-1 chatting
- Unread/Seen Offline Messages
- Send pictures/files
- Store messages in DB
- Message Encryption

### Database schema
- **Conversation Table**

| Conversation_id | User 1 | User 2 | Encryption_key |
|:---------------:|:------:|:------:|:--------------:|
|        5        | User_A | User_B |     AB152C1    |
|        6        | User_A | User_C |     C458A29    |

- **Message Table**

| Conversation_id |  Time |         Text         | From_user | To_user |
|:---------------:|:-----:|:--------------------:|:---------:|:-------:|
|        5        | 12:00 |        "Hi"          |   User_A  | User_B  |
|        6        | 13:04 |      "Hello"         |   User_A  | User_C  |

- **Unread Table**

| From_user | To_user |  time |
|:---------:|:-------:|:-----:|
|   User_A  |  User_C | 13:04 |

- **Seen Table**

| From_user | To_user | time  |
|:---------:|:-------:|:-----:|
|   User_A  |  User_C | 14:09 |



### Design Features

#### User status: Online/ Active [x] hours ago
- User A login through Authentication system
- User A connects bidirectional with Node 2 through Load balancer
    + N2 manages User A
    + User A continously sends heartbeating to N2
    + Info stored in distributed cache and kept updated by N2

| User   | Server | Heartbeat time     |
|:------:|:------:|:------------------:|
| User A | N2     | Mar 13 15:51:22    |
| User B | N3     | Mar 13 13:41:31    |

- Based on the heartbeat time -> user status: online or Active [x] hours ago

```
    User A: Online
    User B: Active 2 hours ago
```
<img src="./img/4.jpg" alt="drawing" width="850"/>

#### Friend Adding

#### Online 1-on-1 chatting
- User A and B are friends
- User A heartbeating N2, User B heartbeating N3

```
    User A: Online
    User B: Online
```

- User A want to send messages to User B
    + User A send message to N2, N2 put the message onto **Message Table**
    + N2 check if any node manages user B -> Found N3
    + N2 send message to N3, N3 sends message to User B


- **Message Table**

| Conversation_id |  Time |         Text         | From_user | To_user |
|:---------------:|:-----:|:--------------------:|:---------:|:-------:|
|        5        | 12:00 |        "Hi"          |   User_A  | User_B  |

#### Offline Messages
- User A and C are friends
- User A heartbeating N2

```
    User A: Online
    User C: Active 2 hrs ago
```

- User A want to send messages to User C offline
    + **13:04** User A send message to N2, N2 put the message onto DB **Message Table**
    + N2 check if any node manages user C -> Not Found
    + N2 puts a note onto **Unread Table**, inform User A message unread
    + **14:09** User C online, heartbeating to N1
    + N1 check **Unread Table**, retrieve the message to User C
    + N1 remove the entry from **Unread Table**, puts a note onto **Seen Table**